[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfc/janus-core/blob/main/docs/source/tutorials/cli/phonons.ipynb)

# Phonons

`janus-core` contains an interface to [phonopy](https://phonopy.github.io/phonopy/) to compute phonons and ready made workflows these calculations.

## Set up environment (optional)

These steps are required to run this tutorial with Google Colab. To do so, uncomment and run the cell below.

This will replace pre-installed versions of `numpy` and `torch` in Colab with versions that are known to be compatible with `janus-core`.

It may be possible to skip the steps that uninstall and reinstall `torch`, which will save a considerable amount of time.

These instructions but may work for other systems too, but it is typically preferable to prepare a virtual environment separately before running this notebook if possible.

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall numpy -y # Uninstall pre-installed numpy

# ! pip uninstall torch torchaudio torchvision transformers -y # Uninstall pre-installed torch
# ! uv pip install torch==2.5.1 # Install pinned version of torch

# ! uv pip install janus-core[mace,visualise] data-tutorials --system # Install janus-core with MACE and WeasWidget, and data-tutorials

# get_ipython().kernel.do_shutdown(restart=True) # Restart kernel to update libraries. This may warn that your session has crashed.

To ensure you have the latest version of `janus-core` installed, compare the output of the following cell to the latest version available at https://pypi.org/project/janus-core/

In [ ]:
from janus_core import __version__

print(__version__)

## Prepare data

Use `data_tutorials` to get the data required for this tutorial:

In [ ]:
from data_tutorials.data import get_data

get_data(
    url="https://raw.githubusercontent.com/stfc/janus-core/main/docs/source/tutorials/data/",
    filename=["NaCl.cif"],
    folder="../data",
)

view the data

In [ ]:
from ase.io import read
from weas_widget import WeasWidget

NaCl = read("../data/NaCl.cif")
print(NaCl.cell)
v=WeasWidget()
v.from_ase(NaCl)
v.avr.model_style = 1
v

## Command-line help and options

Once `janus-core` is installed, the `janus phonons`  command should be available. You may want to execute it in a real terminal to see all the options.

In [ ]:
! janus phonons --help

## simple calculation

start by creating a config for the calculations. In this we performs

- geometry optimisation
- force constants calculations
- band structure (on a path determined by seekpath)
- density of states calculation

In [ ]:
%%writefile phonons.yml

supercell: 2 2 2
displacement: 0.001
fmax: 1.0e-6
bands: True
dos: True
minimize: True
minimize-kwargs:
  write_results: True
arch: mace_mp
model:  medium-mpa-0
write-full: False
plot-to-file: True
device: cpu
tracker: False

now run the calculation itself

In [ ]:
! janus phonons --config phonons.yml --struct ../data/NaCl.cif

explore the results

In [ ]:
!ls janus_results/

In [ ]:
from IPython.display import SVG, display

NaCl_opt = read("janus_results/NaCl-opt.extxyz")
print(NaCl_opt.cell)
display(SVG("janus_results/NaCl-bands.svg"))
display(SVG("janus_results/NaCl-dos.svg"))

## advanced options

we compute thermal properties and also we specify the granularity of the DOS

In [ ]:
%%writefile phonons-adv.yml

supercell: 2 2 2
displacement: 0.001
fmax: 1.0e-6
thermal: True
temp-min: 0
temp-max: 1000
temp-step: 10
bands: True
dos: True
pdos: False
dos-kwargs:
  freq_pitch: 0.05
mesh: [11,11,11]
minimize: True
minimize-kwargs:
  write_results: True
arch: mace_mp
model:  medium-mpa-0
write-full: False
plot-to-file: True
calc-kwargs:
    dispersion: False
    skin: 0 
device: cpu
tracker: False

In [ ]:
! janus phonons --config phonons-adv.yml --struct ../data/NaCl.cif

In [ ]:
!ls janus_results/

In [ ]:
from IPython.display import SVG, display
display(SVG("janus_results/NaCl-bands.svg"))
display(SVG("janus_results/NaCl-dos.svg"))

## phonons under pressure


In [ ]:
%%writefile phonons-pressure.yml

supercell: 2 2 2
displacement: 0.001
fmax: 1.0e-6
thermal: True
temp-min: 0
temp-max: 1000
temp-step: 10
bands: True
dos: True
pdos: False
dos-kwargs:
  freq_pitch: 0.05
mesh: [11,11,11]
minimize: True
minimize-kwargs:
  write_results: True
  filter_kwargs:
      scalar_pressure: 1.0
arch: mace_mp
model:  medium-mpa-0
write-full: False
plot-to-file: True
calc-kwargs:
    dispersion: False
    skin: 0 
device: cpu
tracker: False

In [ ]:
! janus phonons --config phonons-pressure.yml --struct ../data/NaCl.cif

In [ ]:
from IPython.display import SVG, display
display(SVG("janus_results/NaCl-bands.svg"))
display(SVG("janus_results/NaCl-dos.svg"))

## isotropic optimisation of the cell

In [ ]:
%%writefile phonons-iso.yml

supercell: 2 2 2
displacement: 0.001
fmax: 1.0e-6
thermal: True
temp-min: 0
temp-max: 1000
temp-step: 10
bands: True
dos: True
pdos: False
dos-kwargs:
  freq_pitch: 0.05
mesh: [11,11,11]
minimize: True
minimize-kwargs:
  write_results: True
  filter_kwargs:
    hydrostatic_strain: True
arch: mace_mp
model: medium-mpa-0
write-full: False
plot-to-file: True
calc-kwargs:
    dispersion: False
    skin: 0 
device: cpu
tracker: False

In [ ]:
!janus phonons --config phonons-iso.yml --struct ../data/NaCl.cif

In [ ]:
from IPython.display import SVG, display
display(SVG("janus_results/NaCl-bands.svg"))
display(SVG("janus_results/NaCl-dos.svg"))

## using a path for band structure

download mof-5.cif and paths.yaml from examples folder

In [ ]:
get_data(
    url="https://raw.githubusercontent.com/stfc/janus-core/main/docs/source/tutorials/data/",
    filename=["mof-5.cif", "paths.yaml"],
    folder="../data",
)

In [ ]:
mof5 = read("../data/mof-5.cif")
v=WeasWidget()
v.from_ase(mof5)
v.avr.model_style = 1
v

In [ ]:
%%writefile phonons-mof5.yml

supercell: 1 1 1
displacement: 0.001
fmax: 1.0e-6
thermal: True
temp-min: 0
temp-max: 1000
temp-step: 10
bands: True
dos: True
pdos: False
dos-kwargs:
  freq_pitch: 0.05
mesh: [11,11,11]
minimize: True
minimize-kwargs:
  write_results: True
arch: mace_mp
model: medium-mpa-0
write-full: False
plot-to-file: True
calc-kwargs:
    dispersion: False
    skin: 0 
device: cpu
tracker: False

please note parameters here are chosen for speed rather than correctness

In [ ]:
! janus phonons --config phonons-mof5.yml --struct ../data/mof-5.cif

In [ ]:
!ls janus_results/

In [ ]:
from IPython.display import SVG, display
display(SVG("janus_results/mof-5-bs-dos.svg"))

In [ ]:
%%writefile phonons-mof5-path.yml

supercell: 1 1 1
displacement: 0.001
fmax: 1.0e-6
thermal: True
temp-min: 0
temp-max: 1000
temp-step: 10
bands: True
dos: True
pdos: False
dos-kwargs:
  freq_pitch: 0.05
mesh: [11,11,11]
qpoint-file: ../data/paths.yaml
minimize: True
minimize-kwargs:
  write_results: True
arch: mace_mp
model: medium-mpa-0
write-full: False
plot-to-file: True
calc-kwargs:
    dispersion: False
    skin: 0 
device: cpu
tracker: False

In [ ]:
! janus phonons --config phonons-mof5-path.yml --struct ../data/mof-5.cif

In [ ]:
! ls janus_results/

In [ ]:
display(SVG("janus_results/mof-5-bs-dos.svg"))

## extra bits

now as a challenge you can do a more realistic example, change the above example to
-  use a 2 2 2 supercell 
- use a more accurate model for MoFs mof0_v2 from here https://github.com/ddmms/data/tree/main/mace-mof-0/v2

this is part of this paper if you want to know more: https://arxiv.org/abs/2412.02877